In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

geo_derived_feature1.0.json


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 533.7 kB/s eta 0:00:00a 0:00:01


In [4]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [5]:
import os
import paddle
import paddlehub as hub
from paddle.io import Dataset
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
#定义数据集
class myDataset(Dataset):
    def __init__(self):	
        # 数据集存放位置
        self.filepath = "/home/aistudio/data/data273973/derived_place_name.json"  #dataset_dir为数据集实际路径，需要填写全路径
        self.data=pd.read_json(self.filepath)
        self.data.reset_index(drop=True, inplace=True)
    def __getitem__(self, idx):
        toponym=self.data.loc[idx,'dname']
        dfclass=self.data.loc[idx,'dfclass']
        nfclass=self.data.loc[idx,'nfclass']
        label=self.data.loc[idx,'label']
        return {'input':{'toponym':toponym,'dfclass':dfclass,'nfclass':nfclass},'label':label}
    def __len__(self):
        return self.data.shape[0]

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [6]:
from sklearn.model_selection import train_test_split
#构造训练数据集
inputDataset=myDataset()
#分割训练数据集、验证集
trainDataset,valDataset=train_test_split(inputDataset,train_size=0.7,shuffle=True ,random_state=34)
print(len(trainDataset),len(valDataset))

FileNotFoundError: File /home/aistudio/data/data273973/derived_place_name.json does not exist

In [ ]:
from paddlenlp.transformers import BertModel, BertTokenizer,BertForSequenceClassification
from paddlenlp.transformers import ErnieForSequenceClassification,ErnieTokenizer
import paddle.nn as nn
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model =BertForSequenceClassification.from_pretrained('bert-base-uncased')
model=ErnieForSequenceClassification.from_pretrained('ernie-2.0-base-en')
tokenizer=ErnieTokenizer.from_pretrained('ernie-2.0-base-en')
# import paddle.nn as nn
# # 
# class Model(nn.Layer):
#     def __init__(self, model):
#         super(Model, self).__init__()
#         self.bert = model  # allow bert to be config
#         self.dropout = nn.Dropout(p=0.6)
#         self.conv1=nn.Conv1D(768,1,20)
#         self.classifier = nn.Linear(10,2)

#     def forward(self,
#                 input_ids,
#                 token_type_ids=None,
#                 position_ids=None,
#                 attention_mask=None):
#         outputs = self.bert(input_ids,
#                             token_type_ids=token_type_ids,
#                             position_ids=position_ids,
#                             attention_mask=attention_mask)
#         x= paddle.transpose(outputs, perm=[0, 2, 1])
#         x=self.conv1(x)
#         x=F.max_pool1d(x,kernel_size=4)
#         x= paddle.transpose(x, perm=[0, 2, 1])
#         x = self.dropout(x)
#         x=x.reshape([x.shape[0],-1])
#         logits = self.classifier(x)
#         return logits
# #model=Model(model)

In [ ]:
import paddle
import paddle.nn as nn
import paddle.nn.functional as F


class CrossEntropyCriterion(nn.Layer):
    def __init__(self, label_smooth_eps=None,label_num=0):
        """
        标签平滑的交叉熵损失
        输入：
            - label_smooth_eps: 标签平滑的参数
            - label_num: 标签类别数
        """
        super(CrossEntropyCriterion, self).__init__()
        self.label_smooth_eps = label_smooth_eps
        self.label_num= label_num

    def forward(self, predict, label):
        """
        前向计算
        输入：
            - predict: 模型的输出，维度为[批量大小, 标签类别数].
            - label: 标签，维度为[批量大小, 标签类别数]
        """
        # 标签平滑
        #print(F.one_hot(x=label, num_classes=self.label_num))
        label = F.label_smooth(label=F.one_hot(x=label.astype('int32'), num_classes=self.label_num),\
                                   epsilon=self.label_smooth_eps)
        # 经过标签平滑后，label的维度为[批量大小, 序列长度, 目标语言词典大小]
        # 交叉熵损失
        loss = F.cross_entropy(
            input=predict,
            label=label,
            reduction='none',
            soft_label=True if self.label_smooth_eps else False)
        # 返回 总损失，平均损失，非填充词元的数目
        return loss.mean()

In [ ]:
from paddlenlp.data import Dict, Stack, Pad
from paddle.io import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from visualdl import LogWriter
import warnings
warnings.filterwarnings('ignore')
import numpy as np

stack= Stack()
#定义模型训练器
class Trainer():
    def __init__(self,config):
        self.config=config
        self.input_maxlen=config['input_maxlen']  
        self.label_maxlen=config['label_maxlen']
        self.model_path=config['model_path']
        self.model_savepath=config['model_savepath']
        self.resume=config['resume']
        self.log_path=config['log_path']  
        self.best_score=0
        # self.prompt=config['prompt']
        self.model=config['model']
        self.tokenizer=config['tokenizer']
        self.optim=paddle.optimizer.AdamW(learning_rate=config['learning_rate'], \
                                        parameters=model.parameters(), \
                                        weight_decay=config['weight_decay'],\
                                         apply_decay_param_fun=lambda x: x in config['decay_params']) 
        #数据加载
        self.train_dataloader=DataLoader(config['train_dataset'], batch_size=config['batch_size'],shuffle=True,drop_last=True,collate_fn=self.Collate_fn,num_workers=1)
        self.val_dataloader=DataLoader(config['val_dataset'], batch_size=config['batch_size'],shuffle=True,drop_last=True,collate_fn=self.Collate_fn,num_workers=1)
        self.loss_fct = CrossEntropyCriterion(0.4,2)
        #定义
        self.scheduler=paddle.optimizer.lr.LinearWarmup(learning_rate=config['learning_rate'], warmup_steps=len(self.train_dataloader), start_lr=0, end_lr=5e-5,verbose=True) 
    def Collate_fn(self,batch):     
         #对输入字符串进行分词编码
         input_ids_list=[]
         attention_mask_list=[]
         token_type_list=[]
         pred_atten_list=[]
         label_list=[]
         for batch_dict in batch:
              toponym=batch_dict['input']['toponym']
              dfclass=batch_dict['input']['dfclass']
              nfclass=batch_dict['input']['nfclass']
              label=batch_dict['label']
              #####test0######
            #   inputs1='the place name is:{}.'.format(toponym)
            #   inputs2=''
              #########test1#####
              #inputs1='the place name is:{}.'.format(toponym)
              #inputs2='the category of derived place name:{}.'.format(dfclass)
              ########test2#######
              inputs1='the place name is:{}.'.format(toponym)
              inputs2='the category of derived place name:{}.the category of native place name:{}.'.format(dfclass,nfclass)
              input_dict=self.tokenizer(text=[inputs1],text_pair=[inputs2],pad=True,\
                                  add_special_tokens=True,\
                              max_seq_len=self.input_maxlen, \
                              pad_to_max_seq_len=True,truncation=True,return_attention_mask=True)
    
              pred_inputs=paddle.to_tensor(input_dict['input_ids'][0])
              label_ids=paddle.to_tensor(label)
              attention_mask=paddle.to_tensor(input_dict['attention_mask'][0])
              token_type_id=paddle.to_tensor(input_dict['token_type_ids'][0])
              input_ids_list.append(pred_inputs)
              attention_mask_list.append(attention_mask)
              token_type_list.append(token_type_id)
              label_list.append(label_ids)
         #拼接成一整张输入张量
         input_ids_list=stack(input_ids_list)
         attention_mask_list=stack(attention_mask_list)
         label_list=stack(label_list)
         return  {'input_ids':input_ids_list,'attention_mask':attention_mask_list,\
                                     'token_type_ids':token_type_list,'labels':label_list}

    def load_model_optim_scheduler(self):
        print('模型已加载！')
        check_point_dict=paddle.load(os.path.join(self.model_savepath,'best_score.pkl'))
        print(check_point_dict)
        #self.best_score=check_point_dict['best_score']
        self.start_epoch=check_point_dict['metric']['epoch']
        self.model.set_state_dict(paddle.load(os.path.join(self.model_savepath,'model.pdparams')))
        #self.optim.set_state_dict(paddle.load(os.path.join(self.model_savepath,'optim.pdopt')))
        
        return 
    def save_check_point(self,metric):
        # paddle.save(self.model.state_dict(),os.path.join(self.model_savepath,'model.pdparams'))
        self.model.train()
        print('模型已保存！')
        # 保存Layer参数
        paddle.save(self.model.state_dict(), os.path.join(self.model_savepath,"model.pdparams"))
        # 保存优化器参数
        paddle.save(self.optim.state_dict(), os.path.join(self.model_savepath,'optim.pdopt'))
        # 保存检查点checkpoint信息
        paddle.save({'metric':metric},os.path.join(self.model_savepath,'best_score.pkl'))
        return
    def compute_loss(self,input_dict):
        inputs={
             'input_ids':input_dict['input_ids'],
             'attention_mask':input_dict['attention_mask']
                }
        logits=self.model(**inputs)
        labels=input_dict['labels'].reshape([-1])
        loss=self.loss_fct(logits,labels)
        return loss,logits
        
    def compute_metrics(self,input_dict):
        __,logits=self.compute_loss(input_dict)
        logits=logits.reshape([-1, logits.shape[-1]])
        preds=paddle.argmax(logits,axis=-1)
        label_ids=input_dict['labels'].reshape([-1])
        #计算评价指标
        accuracy = accuracy_score(label_ids.cpu(), preds.cpu())
        precision = precision_score(label_ids.cpu(), preds.cpu(), average='weighted')
        recall = recall_score(label_ids.cpu(), preds.cpu(), average='weighted')
        f1 = f1_score(label_ids.cpu(), preds.cpu(), average='weighted') 
        return  accuracy,precision,recall,f1
    def model_train_eval(self):
        if self.resume:
           self.load_model_optim_scheduler()
        #初始化一个记录器
        with LogWriter(logdir=self.log_path) as writer:
            for epoch in range(self.config['epoch']):
                #模型批次训练
                loss_batch=self.model_batch_train()
                # #更新学习率
                self.scheduler.step()
                print('epoch',epoch,"train/loss",loss_batch)
                writer.add_scalar(tag="train/loss", step=epoch, value=loss_batch)
                # # #模型批次评估
                acc,pre,rec,f1=self.model_batch_eval()
                writer.add_scalar(tag="acc", step=epoch, value=acc)
                writer.add_scalar(tag="pre", step=epoch, value=pre)
                writer.add_scalar(tag="rec", step=epoch, value=rec)
                writer.add_scalar(tag="f1", step=epoch, value=f1)
                if  self.best_score<f1:
                    self.best_score=f1
                    metric={}
                    metric['accuracy']= acc
                    metric['precision']=pre
                    metric['recall']=rec
                    metric['f1']=f1
                    metric['epoch']=epoch
                    print('epoch',metric)
                    self.save_check_point(metric)

        return 
    def model_batch_train(self):
        # 设置训练模式
        self.model.train()
        loss_batch=0
        for batch_id,input_dict in enumerate(self.train_dataloader):
            loss,logits=self.compute_loss(input_dict)
            loss_batch+=loss.item()
            #梯度清零
            self.optim.clear_grad()
            #计算梯度，反向传播
            loss.backward()
            #更新参数
            self.optim.step()
        loss_batch/=(batch_id+1)
        return  loss_batch
    @paddle.no_grad()
    def model_batch_eval(self):
        #设置评估模式
        self.model.eval()
        all_preds=[]
        all_labels=[]
        acc_eval,pre_eval,rec_eval,f1_eval=0,0,0,0
        for batch_id, input_dict in enumerate(self.val_dataloader):
             accuracy,precision,recall,f1=self.compute_metrics(input_dict)
             acc_eval+=accuracy
             pre_eval+=precision
             rec_eval+=recall
             f1_eval+=f1
        return  acc_eval/(batch_id+1),pre_eval/(batch_id+1),rec_eval/(batch_id+1),f1_eval/(batch_id+1)

In [ ]:
#模型训练配置
config={
        'model_savepath':r"/home/aistudio/work/ernie_prompt2",
        'log_path':'./log/ernie_prompt2',
        # 指定优化策略，更新模型参数
        'decay_params':[p.name for n, p in model.named_parameters() if not any(nd in n for nd in ["bias", "norm"])],
        'model_path':r'/home/aistudio/work/output',
        #'prompt':'the place name is:{}.',#test0
        #'prompt':'the category of place name:{}.the place name is:{}.',#test2
        'input_maxlen':60,
        'label_maxlen':10,
        'resume':False,
        'model':model,
        'tokenizer':tokenizer,
         'epoch':50,
         'batch_size':128,
         'learning_rate':5e-8,
         'weight_decay':0.1,
         'train_dataset':trainDataset,
         'val_dataset':valDataset}
#训练器初始化
model_trainer=Trainer(config)
model_trainer.model_train_eval()

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
import paddle
model_savepath=r"/home/aistudio/work/ernie_prompt2"
check_point_dict=paddle.load(os.path.join(model_savepath,'best_score.pkl'))
print(check_point_dict)